In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GenAI para análise de Google Trends no BigQuery

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/intro_palm_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Execute no Vertex AI Workbench
    </a>
  </td>
</table>

## Visão geral

O Google Trends é a ferramenta do Google que auxilia usuários e desenvolvedores a terem uma visão do que acontece no mundo através dos padrões de busca, incluindo os termos buscados, no google.com. Neste laboratório vamos utilizar os dados do Google Trends, através do BigQuery, para obter informações sobre algumas dessas tendêncas de buscas.

### BigQuery

O [BigQuery](https://cloud.google.com/bigquery/). é o serviço de base de dados gerenciado, serverless e escalável disponível na Google Cloud. O BigQuery permite gerenciar e analisar dados com recursos integrados, como *machine learning*, análise geoespacial e *business intelligence*. Para maiores detalhes, visite a [documentação oficial](https://cloud.google.com/bigquery/docs) do BigQuery.

### Vertex AI PaLM API
A Vertex AI PaLM API, [lançada em 10 de maio de 2023](https://cloud.google.com/vertex-ai/docs/generative-ai/release-notes#may_10_2023), é desenvolvida com [PaLM 2]( https://ai.google/discover/palm2).

### Usando a API Vertex AI PaLM

Você pode interagir com a API Vertex AI PaLM usando os seguintes métodos:

* Use a UI da [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) para testes rápidos e geração de comandos.
* Faça chamadas REST no Cloud Shell.
* Use o Python SDK em um notebook Jupyter

Este notebook se concentra no uso do Python SDK para chamar a Vertex AI PaLM API. Para obter mais informações sobre como usar o Vertex AI Studio sem escrever código, você pode explorar [Introdução às instruções da interface do usuário](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/getting-started/getting_started_ui.md)

Para obter mais informações, confira a [documentação sobre suporte de IA generativa para Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).

### Objetivos

Neste tutorial, você irá utilizar os serviços de Google Cloud para:

* Explorar dados do dataset público `bigquery-public-data.google_trends`
* Utilizar as API do GenAI Studio para a extração de informações contextuais sobre as tendências de busca

### Custos
Este tutorial usa os seguintes componentes de Google Cloud:

* Vertex AI Studio
* BigQuery

Saiba mais sobre possíveis custos envolvidos [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing),
[preços do BigQuery](https://cloud.google.com/bigquery/pricing),
e use a [Calculadora de preços](https://cloud.google.com/products/calculator/)
para gerar uma estimativa de custo com base no uso projetado.

### Segurança de dados
**P: O Google usa dados de clientes para melhorar seus modelos de base?**
R: Não, o Google não usa dados de clientes para melhorar os modelos de fundação. Os dados do cliente são usados apenas para gerar uma resposta do modelo.

**P: Os funcionários do Google veem os dados que envio ao modelo?**
R: Não, os funcionários do Google não têm acesso aos dados do cliente e todos os dados são criptografados em trânsito, em uso e em repouso.

**P: O Google armazena algum dos dados do cliente que são enviados para o modelo?**
R: Não, o Google não armazena dados de clientes. No entanto, o Google pode armazenar em cache temporariamente os dados do cliente durante a solicitação, como pipeline de ajuste de prompt e uso em batch.

**P: O Google registra dados?**
R: Não, o Google não loga os dados dos clientes. Os logs do lado do sistema ajudam o Google a garantir a integridade e a disponibilidade do sistema.

### IA Responsável
LLMs podem traduzir linguagem, resumir texto, gerar escrita criativa, gerar código, chatbots e assistentes virtuais, além de complementar mecanismos de pesquisa e sistemas de recomendação. Ao mesmo tempo, como uma tecnologia em estágio inicial, seus recursos e usos em evolução podem criar aplicações incorretas, uso indevido e consequências não intencionais ou imprevistas. LLMs podem gerar resultados inesperados, incluindo texto ofensivo, insensível ou incorreto.

Além disso, a incrível versatilidade dos LLMs também é o que torna difícil prever exatamente que tipos de saídas não intencionais ou imprevistas eles podem produzir. Dados esses riscos e complexidades, a API PaLM foi projetada com os [AI Principles do Google](https://ai.google/principles/) em mente. No entanto, é importante que os desenvolvedores entendam e testem seus modelos para implantá-los com segurança e responsabilidade. Para ajudar os desenvolvedores, o Vertex AI Studio possui filtragem de conteúdo integrada e a API PaLM possui pontuação de atributo de segurança para ajudar os clientes a testar os filtros de segurança do Google e definir limites de confiança adequados para seu caso de uso e negócios. Consulte a seção [Filtros e atributos de segurança](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai#safety_filters_and_attributes) para saber mais.

Quando a API PaLM é integrada ao caso de uso e contexto exclusivos de um cliente, considerações adicionais de IA Responsável e [limitações PaLM](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai #palm_limitations) precisam ser considerados. Incentivamos os clientes a usar *fairness*, interpretabilidade, privacidade e segurança [práticas recomendadas](https://ai.google/responsabilidades/responsible-ai-practices/).

## Primeiros Passos

### Instalando os SDK da Vertex AI

In [ ]:
!pip install google-cloud-aiplatform langchain chromadb flask-sqlalchemy --upgrade --user

**Somente Colab:** Descomente a célula a seguir para reiniciar o kernel ou use o botão para reiniciar o kernel.

In [ ]:
# # Reinicia automaticamente o kernel após as instalações para que seu ambiente possa acessar os novos pacotes
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Autenticando seu ambiente de notebook
* Se você estiver usando o **Colab** para executar este notebook, descomente a célula abaixo e continue.
* Se você estiver usando o **Vertex AI Workbench**, confira as instruções de configuração [aqui](../setup-env/README.md).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Importando as bibliotecas necessárias

**Somente Colab:** Descomente a célula a seguir para realizar o processo adequado de inicialização da SDK da Vertex AI.  

In [ ]:
# import vertexai

# PROJECT_ID = "[seu-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

import sys
import pandas as pd
from typing import Union
from google.cloud import bigquery
from vertexai import language_models
from langchain.llms.base import LLM
from datetime import datetime, timedelta
from langchain.embeddings.base import Embeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders.dataframe import DataFrameLoader
from vertexai.preview.language_models import TextGenerationModel

from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.embeddings.base import Embeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains.summarize import load_summarize_chain

## Geração de texto com `text-bison@001`

O modelo de geração de texto da API PaLM que você usará neste notebook é `text-bison@001`. Já deixaremos seu objeto instanciado neste Jupyter notebook para uso futuro

#### Carregando o modelo `text-bison`

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Preparando as funções necessárias 

In [ ]:
def rate_limit(max_per_minute):
  period = 60 / max_per_minute
  while True:
    before = time.time()
    yield
    after = time.time()
    elapsed = after - before
    sleep_time = max(0, period - elapsed)
    if sleep_time > 0:
      print(f'Sleeping {sleep_time:.1f} seconds')
      time.sleep(sleep_time)


class VertexEmbeddings(Embeddings):

  def __init__(self, model, *, requests_per_minute=15):
    self.model = model
    self.requests_per_minute = requests_per_minute

  def embed_documents(self, texts):
    limiter = rate_limit(self.requests_per_minute)
    results = []
    docs = list(texts)

    while docs:
      # Working in batches of 2 because the API apparently won't let
      # us send more than 2 documents per request to get embeddings.
      head, docs = docs[:2], docs[2:]
      chunk = self.model.get_embeddings(head)
      results.extend(chunk)
      next(limiter)

    return [r.values for r in results]

  def embed_query(self, text):
    single_result = self.embed_documents([text])
    return single_result[0]


class VertexLLM(LLM):

  model: language_models.TextGenerationModel
  predict_kwargs: dict

  def __init__(self, model, **predict_kwargs):
    super().__init__(model=model, predict_kwargs=predict_kwargs)

  @property
  def _llm_type(self):
    return 'vertex'

  def _call(self, prompt, stop=None):
    result = self.model.predict(prompt, **self.predict_kwargs)
    return str(result)

  @property
  def _identifying_params(self):
    return {}

REQUESTS_PER_MINUTE = 6000

model = language_models.TextGenerationModel.from_pretrained('google/text-bison@001')
llm = VertexLLM(
  model,
  max_output_tokens=256,
  temperature=0,
  top_p=0.8,
  top_k=40
)
embedding = VertexEmbeddings(language_models.TextEmbeddingModel.from_pretrained('google/textembedding-gecko@001'), requests_per_minute=REQUESTS_PER_MINUTE)

llm

### Criação do dataset no BigQuery

Para termos uma estrutura pronta no projeto do laboratório para salvar e explorar dados, vamos criar um projeto no BigQuery:

In [ ]:
project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id

client = bigquery.Client(project=project_id)
dataset_id = f"{project_id}.google_trends"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
try:
    dataset = client.create_dataset(dataset, timeout=30)
    print("Dataset criado com sucesso {}.{}".format(project_id, dataset.dataset_id))
except:
    print(f"O dataset {dataset_id} já existe.")

### Visualizando os dados de Google Trends

Os dados de Google Trends, ou seja os termos de busca mais utilizados em determinado dia/semana, são partes de um dataset público do BigQuery chamado `bigquery-public-data.google_trends`. Aqui vamos visualizar os dados disponíveis das duas últimas semanas e salvar essas informações em um Pandas DataFrame:

In [ ]:
query = """
SELECT
 DISTINCT term AS Top_Term,
 refresh_date AS Day,
 rank
FROM `bigquery-public-data.google_trends.international_top_terms`
WHERE
 rank <= 3
AND country_code = "BR"
 # AND refresh_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 2 WEEK)
GROUP BY Day, Top_Term, rank
ORDER BY Day DESC, rank;
"""

client = bigquery.Client(project=project_id)
df = client.query(query).to_dataframe()
df

### Reailzando explorações dos dados utilizando as API do Vertex AI Studio

Com os dados disponíveis em um dataframe, vamos primeiro utilizar as 5 (cinco) primeiras linhas do DataFrame para fazer uma experimentação rápida:

In [ ]:
anteontem = datetime.now() - timedelta(2)
anteontem = anteontem.strftime('%Y-%m-%d')
    
googletrends_table = df.head(5).to_markdown()

prompt = (f"""
  Use a tabela a seguir, em formato markdown, para responder a questão seguinte.
  Considere que a coluna 'Top_Term' representa o termo de busca.
  Considere que a coluna 'Day' representa o dia, em formato 'YYYY-MM-DD'.
  Considere que a coluna 'rank' representa a posição daquele termo naquele dia.

  {googletrends_table}

  Gere uma sentença que descreva as top-3 trends de {anteontem}
""")

print(model.predict(prompt).text)

### Criando um dataframe com todo o conteúdo de Google Trends das últimas 2 semanas

Com a experimentação realizada com as 5 (cinco) primeiras linhas, agora podemos seguir em frente e criar um dataframe que possua todos os top-3 das últmas duas semanas:

In [ ]:
import pandas as pd

data = []
for index, rows in df.iterrows():
  text = """No dia {0}, o top {1} termo mais buscado no Google foi {2}.
  """.format(
      rows['Day'],
      rows['rank'],
      rows['Top_Term'])
  data.append(text)

nl_googletrends_df = pd.DataFrame(data, columns=['text'])
nl_googletrends_df

### Utilizando LangChain para criar uma estrutura de busca de informações

Neste próximo passo vamos utilizar a biblioteca LangChain para gerar embeddings de todas as informações do DataFrame e depois armazenar como índices de busca dentro do diretório `dados-locais/`:

In [ ]:
persistent_path = './dados-locais'
df_loader = DataFrameLoader(nl_googletrends_df, page_content_column="text")

googletrends_index = VectorstoreIndexCreator(
    embedding=embedding, 
    vectorstore_kwargs={
        'persist_directory': f'{persistent_path}/googletrends'
    }).from_loaders([df_loader])

In [ ]:
def ask_question(pergunta, max_results=5, threshold=0.5, context=True):
    similar_docs = googletrends_index.vectorstore.similarity_search_with_score(pergunta, llm=llm, k=max_results)
    filtered_docs = list(filter(lambda doc: doc[1] <= threshold, similar_docs))
    context = "\n".join([doc.page_content for doc, score in filtered_docs])
    prompt = (f"""
    
    {context}
    
     Responda com base apenas nos dados de contexto.
     Escreva uma frase sucinta algo interessante sobre o termo de pesquisa.
     

     Pergunta: {pergunta}
     Resposta:    """)
    resposta = model.predict(prompt)
    return(resposta.text)

### Realizando perguntas em linguagem natural

Agora, considerando as informações de Google Trends das últimas duas semanas, podemos realizar perguntas, em linguagem natural, e buscar as informações obtidas a partir do BigQuery e que foram processadas pelo LangChain:

In [ ]:
anteontem = datetime.now() - timedelta(2)
anteontem = anteontem.strftime('%Y-%m-%d')
print("anteontem:", str(anteontem))

print(ask_question(f"Qual é o top-1 termo de pesquisa em '{anteontem}'?"))

In [ ]:
antes_anteontem = datetime.now() - timedelta(3)
antes_anteontem = antes_anteontem.strftime('%Y-%m-%d')
print("antes de anteontem:", str(antes_anteontem))

print(ask_question(f"Qual é o top-3 termo de pesquisa de '{antes_anteontem}'?"))

In [ ]:
ground_truth = df.sample()

print(ask_question(f'Quando que a busca pelo termo {ground_truth.Top_Term} estava no top-3 de termos de busca?'))
print()
print(f"Validação da informação consultando o dataframe de top terms:")
ground_truth